# BATTLE OF THE NEIGHBORHOODS PROJECT
## For the Coursera IBM Data Science Course Capstone Project (Week 2 Assignment) 

## Background of the Problem Statement

<p>Building a good chain of outlets for any buisness is a necessity in the current world to survive as a brand. Many brands are unable to sustain in the market even after good quality of products and services for just one most important reason, which is, bad placement of the outlet/branch or not expanding to the correct location at the correct time.</p>
    <p>Correct time of expansion depends upon the brand, which usually depends on the quality or products and services as good quality of products and services gain them the necessary fundings to expand. After that, it's upon the brand to invest their resources on expansion or modification of existing outlets.</p>
    <p>Though modifications of current outlet/branch is a good step, but in most of the cases, in contrast to expansion, it's effects on the profits is very less.</p>
    <p>Correct placement of the outlet/branch in a given neighborhood is a very important step which must be done with all the necessary background studies done as one wrong placement can result into huge loss, and thus we decided to deal with this particular problem. Our area of concern for this project will be the state of New York.</p>
    

## Problem Statement

Keeping in mind the problem stated in the background study, and for a sample client in our scenario, i.e., a Pizza Place owner. Thus the problem statement can be stated as:<br>
**"To find the best locations in New York State for the expansion of a Pizza Place based in Carnegie Hill, Manhattan, NY."**

## Data Description

<p>Data with us is in the form of a JSON file which contains all the boroughs and their neighborhoods in New York State and their locations in the form of latitude and longitude.</p>
<p> We will also be leveraging the foursquare API to retrieve all the common venues in every neighborhood, which returns the data of all nearby venues (within specified range)(name and LatLng values) in the form of a JSON file.</p>

## Data Usage

<p> Data retrieved will be used to find all nearby venues for each neighborhood and then the retrieved data will be evaluated to find which areas do not have sufficient number of such places and/or sufficient number of venues in the area.</p>
<p> Result will be shown to the client as which areas will be best in New York state to expand the buisness.</p>

# ----------------------------------------------Project Code--------------------------------------------

## Importing all required dependencies

In [1]:
import json
import pandas as pd
import numpy as np
import geopy
import requests
import folium
from geopy import Nominatim

#### Load the JSON dataset of New York state

Load the dataset

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Look into the dataset for further use

In [3]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time and quickly examine through the dataFrame created and make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Let's explore all boroughs in our DataFrame

In [12]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [13]:
from matplotlib import cm
neighborhoods_numbers = neighborhoods['Borough'].value_counts().rename_axis('Borough').reset_index(name='counts')
neighborhoods_numbers = neighborhoods_numbers.set_index('Borough')
color = cm.inferno_r(np.linspace(.4, .8, 30))
barPlot = neighborhoods_numbers.plot.bar(y='counts', figsize=(5,5), title='Number of Neighbourhoods in each Borough', rot=45, legend=False, color=color)
barPlot.set_ylabel('Number of Neighbourhoods')
barPlot
print(neighborhoods_numbers)

               counts
Borough              
Queens             81
Brooklyn           70
Staten Island      63
Bronx              52
Manhattan          40


We have 5 different Boroughs in the state of New York which can be analysed to select the best location for our client to expand their buisness.

Let's divide each borough into separate DataFrames for easier ananlysis of neighborhoods in each borough.

In [14]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [15]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [16]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [17]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [18]:
staten_island_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
staten_island_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


#### Create a map of New York with neighborhoods superimposed on top.


In [19]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Borough'], bronx_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)
for lat, lng, borough, neighborhood in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Borough'], brooklyn_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)
for lat, lng, borough, neighborhood in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Borough'], queens_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)
for lat, lng, borough, neighborhood in zip(staten_island_data['Latitude'], staten_island_data['Longitude'], staten_island_data['Borough'], staten_island_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='cadetblue',
        fill=True,
        fill_color='cadetblue',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)
    
map_newyork

First, let's create a function for the GET request URL. Name your URL **url** and also set the search radius to 500m and maximum limit of number of places as 100.

In [20]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we run the above function on each neighborhood and create a new dataframe called ">>Borough_Name<<_venues."

In [21]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )
staten_island_venues = getNearbyVenues(names=staten_island_data['Neighborhood'],
                                   latitudes=staten_island_data['Latitude'],
                                   longitudes=staten_island_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights
Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery 

Let's have a look at the recently made nearby venues tables.

In [22]:
bronx_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [23]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [24]:
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore


In [25]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [26]:
staten_island_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
3,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
4,St. George,40.644982,-74.079353,Shake Shack,40.643660,-74.075891,Burger Joint


We know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Let's check the sizes of the resulting dataframes

In [28]:
print('Bronx Venues DataFrame size: ', bronx_venues.shape)
print('Manhattan Venues DataFrame size: ', manhattan_venues.shape)
print('Brooklyn Venues DataFrame size: ', brooklyn_venues.shape)
print('Queens Venues DataFrame size: ', queens_venues.shape)
print('Staten Islands Venues DataFrame size: ', staten_island_venues.shape)

Bronx Venues DataFrame size:  (1177, 7)
Manhattan Venues DataFrame size:  (3093, 7)
Brooklyn Venues DataFrame size:  (2744, 7)
Queens Venues DataFrame size:  (2083, 7)
Staten Islands Venues DataFrame size:  (806, 7)


Let's check how many venues were returned for each neighborhood in each borough

In [29]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,27,27,27,27,27,27
Baychester,20,20,20,20,20,20
Bedford Park,37,37,37,37,37,37
Belmont,98,98,98,98,98,98
Bronxdale,13,13,13,13,13,13
Castle Hill,7,7,7,7,7,7
City Island,28,28,28,28,28,28
Claremont Village,17,17,17,17,17,17
Clason Point,9,9,9,9,9,9


In [30]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,71,71,71,71,71,71
Carnegie Hill,85,85,85,85,85,85
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,93,93,93,93,93,93
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [31]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,48,48,48,48,48,48
Bay Ridge,88,88,88,88,88,88
Bedford Stuyvesant,26,26,26,26,26,26
Bensonhurst,30,30,30,30,30,30
Bergen Beach,6,6,6,6,6,6
Boerum Hill,92,92,92,92,92,92
Borough Park,20,20,20,20,20,20
Brighton Beach,44,44,44,44,44,44
Broadway Junction,18,18,18,18,18,18


In [32]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,18,18,18,18,18,18
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,18,18,18,18,18,18
Bay Terrace,37,37,37,37,37,37
Bayside,75,75,75,75,75,75
Bayswater,2,2,2,2,2,2
Beechhurst,16,16,16,16,16,16
Bellaire,13,13,13,13,13,13


In [33]:
staten_island_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,8,8,8,8,8,8
Arden Heights,4,4,4,4,4,4
Arlington,5,5,5,5,5,5
Arrochar,19,19,19,19,19,19
Bay Terrace,11,11,11,11,11,11
Bloomfield,5,5,5,5,5,5
Bulls Head,45,45,45,45,45,45
Butler Manor,6,6,6,6,6,6
Castleton Corners,17,17,17,17,17,17


<strong>Note</strong>: Number of venues in each neighborhood is a very important factor for our project as neighborhoods with less number of venues are very open to new buisness as compared to neighborhoods with more number of venues.

### Now we will analyze each neighborhood in each borough to find which neighborhoods will make a good spot to open a new chain for our client.

Analysis of Bronx Borough

In [34]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name = 'Neighborhood'
neighborhood_col = bronx_onehot.pop(col_name)
bronx_onehot.insert(0, col_name, neighborhood_col)

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,...,Train,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name = 'Neighborhood'
neighborhood_col = manhattan_onehot.pop(col_name)
manhattan_onehot.insert(0, col_name, neighborhood_col)

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name = 'Neighborhood'
neighborhood_col = brooklyn_onehot.pop(col_name)
brooklyn_onehot.insert(0, col_name, neighborhood_col)

brooklyn_onehot.head()

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name = 'Neighborhood'
neighborhood_col = queens_onehot.pop(col_name)
queens_onehot.insert(0, col_name, neighborhood_col)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# one hot encoding
staten_island_onehot = pd.get_dummies(staten_island_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
staten_island_onehot['Neighborhood'] = staten_island_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name = 'Neighborhood'
neighborhood_col = staten_island_onehot.pop(col_name)
staten_island_onehot.insert(0, col_name, neighborhood_col)

staten_island_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,...,Train,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store,Yoga Studio
0,Allerton,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,Baychester,0.000000,0.000000,0.050000,0.05,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,Bedford Park,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.027027,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,Belmont,0.000000,0.000000,0.010204,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.010204,0.000000,0.00000
4,Bronxdale,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
5,Castle Hill,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,City Island,0.000000,0.000000,0.035714,0.00,0.000000,0.035714,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
7,Claremont Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
8,Clason Point,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
9,Co-op City,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [40]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.014085,0.028169,0.000,0.014085,0.000000
1,Carnegie Hill,0.000000,0.000000,0.00,0.000000,0.011765,0.000000,0.000000,0.000000,0.011765,...,0.00,0.023529,0.000000,0.000000,0.000000,0.011765,0.035294,0.000,0.000000,0.035294
2,Central Harlem,0.000000,0.000000,0.00,0.068182,0.045455,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000,0.010000,0.000000
4,Chinatown,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.010000
5,Civic Center,0.000000,0.000000,0.00,0.000000,0.053763,0.010753,0.000000,0.000000,0.000000,...,0.00,0.010753,0.000000,0.000000,0.010753,0.010753,0.021505,0.000,0.000000,0.032258
6,Clinton,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.030000,0.000,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,...,0.00,0.020000,0.000000,0.000000,0.000000,0.030000,0.010000,0.000,0.000000,0.000000
9,Financial District,0.000000,0.000000,0.00,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000,0.000000,0.010000


In [41]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bath Beach,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bay Ridge,0.000000,0.00,0.034091,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.011364,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bedford Stuyvesant,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.000000,0.000000
3,Bensonhurst,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bergen Beach,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Boerum Hill,0.000000,0.00,0.010870,0.00,0.010870,0.00,0.000000,0.000000,0.032609,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.021739
6,Borough Park,0.000000,0.00,0.050000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Brighton Beach,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Broadway Junction,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Brooklyn Heights,0.000000,0.00,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.020000,0.000000,0.010000,0.040000


In [42]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000
1,Astoria,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.0,0.0,...,0.010000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.010000,0.000000,0.000000
2,Astoria Heights,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Auburndale,0.000000,0.000000,0.000000,0.055556,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bay Terrace,0.027027,0.000000,0.000000,0.054054,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.027027,0.000000,0.000000,0.0,0.027027,0.000000,0.000000,0.054054,0.000000
5,Bayside,0.000000,0.000000,0.000000,0.040000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.013333,0.0,0.000000,0.013333,0.000000,0.000000,0.013333
6,Bayswater,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,Beechhurst,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.062500
8,Bellaire,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,Belle Harbor,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [43]:
staten_island_grouped = staten_island_onehot.groupby('Neighborhood').mean().reset_index()
staten_island_grouped

,Neighborhood,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,Annadale,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.125000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Arden Heights,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,Arlington,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Arrochar,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.052632,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,Bay Terrace,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.090909,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
5,Bloomfield,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
6,Bulls Head,0.000000,0.022222,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.022222,0.00,0.022222,0.000000
7,Butler Manor,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8,Castleton Corners,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
9,Charleston,0.000000,0.033333,0.00,0.000000,0.0,0.033333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.000000


### Let's find the most common venues in each borough for each neighborhood for a better understanding of each neighborhood

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Most Common Venues for Bronx Borough

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
bronx_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
bronx_neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    bronx_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

bronx_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega,Donut Shop,Breakfast Spot,Pharmacy,Spa,Fast Food Restaurant,Electronics Store,Smoke Shop
1,Baychester,Donut Shop,Pet Store,Bank,Men's Store,Mattress Store,Spanish Restaurant,Fast Food Restaurant,Electronics Store,Supermarket,Fried Chicken Joint
2,Bedford Park,Diner,Pizza Place,Chinese Restaurant,Deli / Bodega,Pharmacy,Mexican Restaurant,Bus Station,Spanish Restaurant,Sandwich Place,Supermarket
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Bank,Donut Shop,Mexican Restaurant,Grocery Store,Dessert Shop,Sandwich Place
4,Bronxdale,Breakfast Spot,Bank,Chinese Restaurant,Performing Arts Venue,Park,Paper / Office Supplies Store,Eastern European Restaurant,Spanish Restaurant,Mexican Restaurant,Deli / Bodega


Most Common Venues for Manhattan Borough

In [46]:
manhattan_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
manhattan_neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    manhattan_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

manhattan_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Boat or Ferry,Plaza,Playground,Shopping Mall,Mexican Restaurant
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Wine Shop,Bookstore,Gym,Bar,Japanese Restaurant,Shipping Store
2,Central Harlem,African Restaurant,Chinese Restaurant,Seafood Restaurant,Cosmetics Shop,French Restaurant,Bar,American Restaurant,Food Truck,Beer Bar,Gym / Fitness Center
3,Chelsea,Art Gallery,Coffee Shop,Café,Bakery,Ice Cream Shop,American Restaurant,Italian Restaurant,Seafood Restaurant,Market,Park
4,Chinatown,Chinese Restaurant,Bakery,Bubble Tea Shop,Cocktail Bar,Coffee Shop,Ice Cream Shop,Salon / Barbershop,American Restaurant,Optical Shop,Vietnamese Restaurant


Most Common Venues for Brooklyn Borough

In [47]:
brooklyn_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
brooklyn_neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brooklyn_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brooklyn_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Pizza Place,Chinese Restaurant,Gas Station,Sushi Restaurant,Italian Restaurant,Asian Restaurant,Bubble Tea Shop,Fast Food Restaurant,Burger Joint
1,Bay Ridge,Spa,Pizza Place,Italian Restaurant,Greek Restaurant,Gym / Fitness Center,Bar,American Restaurant,Middle Eastern Restaurant,Hookah Bar,Ice Cream Shop
2,Bedford Stuyvesant,Pizza Place,Coffee Shop,Café,Bar,Tiki Bar,Deli / Bodega,Discount Store,Park,Cocktail Bar,New American Restaurant
3,Bensonhurst,Sushi Restaurant,Dessert Shop,Ice Cream Shop,Flower Shop,Italian Restaurant,Donut Shop,Bakery,Noodle House,Liquor Store,Cha Chaan Teng
4,Bergen Beach,Harbor / Marina,Donut Shop,Baseball Field,Playground,Athletics & Sports,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Field


Most Common Venues for Queens Borough

In [48]:
queens_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
queens_neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    queens_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

queens_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Board Shop,Bus Stop,Thai Restaurant,Bed & Breakfast,Donut Shop,Coffee Shop,Beach
1,Astoria,Bar,Middle Eastern Restaurant,Mediterranean Restaurant,Greek Restaurant,Hookah Bar,Indian Restaurant,Deli / Bodega,Bakery,Seafood Restaurant,Café
2,Astoria Heights,Playground,Italian Restaurant,Plaza,Bus Station,Bowling Alley,Supermarket,Laundromat,Bakery,Cocktail Bar,Burger Joint
3,Auburndale,Bar,Discount Store,Fast Food Restaurant,Furniture / Home Store,Supermarket,Noodle House,Sushi Restaurant,Korean Restaurant,Miscellaneous Shop,Italian Restaurant
4,Bay Terrace,Clothing Store,Cosmetics Shop,Lingerie Store,Mobile Phone Shop,Shoe Store,Donut Shop,Women's Store,American Restaurant,Kids Store,Movie Theater


Most Common Venues for Staten Island Borough

In [49]:
staten_island_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
staten_island_neighborhoods_venues_sorted['Neighborhood'] = staten_island_grouped['Neighborhood']

for ind in np.arange(staten_island_grouped.shape[0]):
    staten_island_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(staten_island_grouped.iloc[ind, :], num_top_venues)

staten_island_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Park,Sports Bar,Pizza Place,Train Station,Restaurant,Diner,Pharmacy,Food,Eastern European Restaurant,Fish & Chips Shop
1,Arden Heights,Coffee Shop,Pizza Place,Pharmacy,Deli / Bodega,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop
2,Arlington,Bus Stop,Deli / Bodega,Boat or Ferry,Grocery Store,Frame Store,Food Truck,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop
3,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Polish Restaurant,Bagel Shop,Middle Eastern Restaurant,Supermarket,Mediterranean Restaurant,Sandwich Place,Outdoors & Recreation
4,Bay Terrace,Italian Restaurant,Supermarket,Donut Shop,Shipping Store,Home Service,Playground,Train Station,Sushi Restaurant,Salon / Barbershop,Flower Shop


Now we narrow down the neighborhoods where our Venue Category of concern, i.e., Pizza Place, is not in the top 10 most common venues and combine them into one table because if our category of concern is not so common in any area, then the business of our client will have greater chance to succeed since they will be one of a kind in the area and will have higher chances of being liked by the people and will have lesser competition.

In [50]:
suggestedAreas = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'])

Now we will check for all areas in Bronx Borough and add the neighborhoods which do not contain our Venue Category of concern, i.e., Pizza Place, among the top 10 most common venues.

In [51]:
common_venues = []
for i in range(bronx_neighborhoods_venues_sorted.shape[0]):
    for j in range(1, bronx_neighborhoods_venues_sorted.shape[1]):
        common_venues.append(bronx_neighborhoods_venues_sorted.iloc[i][j])
    if "Pizza Place" not in common_venues:
        neigh = bronx_neighborhoods_venues_sorted.iloc[i][0]
        lat = bronx_data.loc[bronx_data['Neighborhood'] == bronx_neighborhoods_venues_sorted.iloc[i][0], 'Latitude'].values[0]
        lng = bronx_data.loc[bronx_data['Neighborhood'] == bronx_neighborhoods_venues_sorted.iloc[i][0], 'Longitude'].values[0]
        suggestedAreas = suggestedAreas.append({'Borough': 'Bronx', 'Neighborhood': neigh, 'Latitude': lat, 'Longitude': lng}, ignore_index=True)
    common_venues=[]
suggestedAreas

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Bronxdale,40.852723,-73.861726
2,Bronx,City Island,40.847247,-73.786488
3,Bronx,Clason Point,40.806551,-73.854144
4,Bronx,Country Club,40.844246,-73.824099
5,Bronx,Eastchester,40.887556,-73.827806
6,Bronx,Edenwald,40.884561,-73.848083
7,Bronx,Fieldston,40.895437,-73.905643
8,Bronx,Longwood,40.815099,-73.895788
9,Bronx,Mount Hope,40.848842,-73.908299


Similarly let's add neighborhoods of all other boroughs in New York state where "Pizza Place" is not among the top 10 most common places.

In [52]:
common_venues = []
for i in range(manhattan_neighborhoods_venues_sorted.shape[0]):
    for j in range(1, manhattan_neighborhoods_venues_sorted.shape[1]):
        common_venues.append(manhattan_neighborhoods_venues_sorted.iloc[i][j])
    if "Pizza Place" not in common_venues:
        neigh = manhattan_neighborhoods_venues_sorted.iloc[i][0]
        lat = manhattan_data.loc[manhattan_data['Neighborhood'] == manhattan_neighborhoods_venues_sorted.iloc[i][0], 'Latitude'].values[0]
        lng = manhattan_data.loc[manhattan_data['Neighborhood'] == manhattan_neighborhoods_venues_sorted.iloc[i][0], 'Longitude'].values[0]
        suggestedAreas = suggestedAreas.append({'Borough': 'Manhattan', 'Neighborhood': neigh, 'Latitude': lat, 'Longitude': lng}, ignore_index=True)
    common_venues=[]
suggestedAreas

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Bronxdale,40.852723,-73.861726
2,Bronx,City Island,40.847247,-73.786488
3,Bronx,Clason Point,40.806551,-73.854144
4,Bronx,Country Club,40.844246,-73.824099
5,Bronx,Eastchester,40.887556,-73.827806
6,Bronx,Edenwald,40.884561,-73.848083
7,Bronx,Fieldston,40.895437,-73.905643
8,Bronx,Longwood,40.815099,-73.895788
9,Bronx,Mount Hope,40.848842,-73.908299


In [53]:
common_venues = []
for i in range(brooklyn_neighborhoods_venues_sorted.shape[0]):
    for j in range(1, brooklyn_neighborhoods_venues_sorted.shape[1]):
        common_venues.append(brooklyn_neighborhoods_venues_sorted.iloc[i][j])
    if "Pizza Place" not in common_venues:
        neigh = brooklyn_neighborhoods_venues_sorted.iloc[i][0]
        lat = brooklyn_data.loc[brooklyn_data['Neighborhood'] == brooklyn_neighborhoods_venues_sorted.iloc[i][0], 'Latitude'].values[0]
        lng = brooklyn_data.loc[brooklyn_data['Neighborhood'] == brooklyn_neighborhoods_venues_sorted.iloc[i][0], 'Longitude'].values[0]
        suggestedAreas = suggestedAreas.append({'Borough': 'Brooklyn', 'Neighborhood': neigh, 'Latitude': lat, 'Longitude': lng}, ignore_index=True)
    common_venues=[]
suggestedAreas

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Bronxdale,40.852723,-73.861726
2,Bronx,City Island,40.847247,-73.786488
3,Bronx,Clason Point,40.806551,-73.854144
4,Bronx,Country Club,40.844246,-73.824099
5,Bronx,Eastchester,40.887556,-73.827806
6,Bronx,Edenwald,40.884561,-73.848083
7,Bronx,Fieldston,40.895437,-73.905643
8,Bronx,Longwood,40.815099,-73.895788
9,Bronx,Mount Hope,40.848842,-73.908299


In [54]:
common_venues = []
for i in range(queens_neighborhoods_venues_sorted.shape[0]):
    for j in range(1, queens_neighborhoods_venues_sorted.shape[1]):
        common_venues.append(queens_neighborhoods_venues_sorted.iloc[i][j])
    if "Pizza Place" not in common_venues:
        neigh = queens_neighborhoods_venues_sorted.iloc[i][0]
        lat = queens_data.loc[queens_data['Neighborhood'] == queens_neighborhoods_venues_sorted.iloc[i][0], 'Latitude'].values[0]
        lng = queens_data.loc[queens_data['Neighborhood'] == queens_neighborhoods_venues_sorted.iloc[i][0], 'Longitude'].values[0]
        suggestedAreas = suggestedAreas.append({'Borough': 'Queens', 'Neighborhood': neigh, 'Latitude': lat, 'Longitude': lng}, ignore_index=True)
    common_venues=[]
suggestedAreas

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Bronxdale,40.852723,-73.861726
2,Bronx,City Island,40.847247,-73.786488
3,Bronx,Clason Point,40.806551,-73.854144
4,Bronx,Country Club,40.844246,-73.824099
5,Bronx,Eastchester,40.887556,-73.827806
6,Bronx,Edenwald,40.884561,-73.848083
7,Bronx,Fieldston,40.895437,-73.905643
8,Bronx,Longwood,40.815099,-73.895788
9,Bronx,Mount Hope,40.848842,-73.908299


In [55]:
common_venues = []
for i in range(staten_island_neighborhoods_venues_sorted.shape[0]):
    for j in range(1, staten_island_neighborhoods_venues_sorted.shape[1]):
        common_venues.append(staten_island_neighborhoods_venues_sorted.iloc[i][j])
    if "Pizza Place" not in common_venues:
        neigh = staten_island_neighborhoods_venues_sorted.iloc[i][0]
        lat = staten_island_data.loc[staten_island_data['Neighborhood'] == staten_island_neighborhoods_venues_sorted.iloc[i][0], 'Latitude'].values[0]
        lng = staten_island_data.loc[staten_island_data['Neighborhood'] == staten_island_neighborhoods_venues_sorted.iloc[i][0], 'Longitude'].values[0]
        suggestedAreas = suggestedAreas.append({'Borough': 'Staten Island', 'Neighborhood': neigh, 'Latitude': lat, 'Longitude': lng}, ignore_index=True)
    common_venues=[]
suggestedAreas

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Bronxdale,40.852723,-73.861726
2,Bronx,City Island,40.847247,-73.786488
3,Bronx,Clason Point,40.806551,-73.854144
4,Bronx,Country Club,40.844246,-73.824099
5,Bronx,Eastchester,40.887556,-73.827806
6,Bronx,Edenwald,40.884561,-73.848083
7,Bronx,Fieldston,40.895437,-73.905643
8,Bronx,Longwood,40.815099,-73.895788
9,Bronx,Mount Hope,40.848842,-73.908299


In [56]:
suggestedAreas.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Bronxdale,40.852723,-73.861726
2,Bronx,City Island,40.847247,-73.786488
3,Bronx,Clason Point,40.806551,-73.854144
4,Bronx,Country Club,40.844246,-73.824099


Now let's plot all these areas on map for better visualization of our gathered and processed data.

In [57]:
# create map of New York using latitude and longitude values
newyork_map = folium.Map(location=[latitude, longitude], zoom_start=10, width=900, height=500)

# add markers to map
for lat, lng, borough, neighborhood in zip(suggestedAreas['Latitude'], suggestedAreas['Longitude'], suggestedAreas['Borough'], suggestedAreas['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=500,
        color=None,
        fill=True,
        fill_color='green',
        fill_opacity=0.8,
        parse_html=False).add_to(newyork_map)
    folium.Circle(
        [lat, lng],
        radius=1000,
        popup=label,
        color=None,
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(newyork_map) 
    
newyork_map

Let's make a dataframe consisting of all Pizza Places and their LatLng values, that are in New York state, and cluster them based on their density to find which areas have lesser number of pizza places.

In [58]:
pizzaPlaces = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Venue', 'Venue Latitude', 'Venue Longitude'])

In [59]:
columns = ['Borough', 'Neighborhood', 'Venue', 'Venue Latitude', 'Venue Longitude']
df = bronx_venues[bronx_venues['Venue Category'] == 'Pizza Place']
df['Borough'] = 'Bronx'
df = df[columns]
pizzaPlaces = pizzaPlaces.append(df, ignore_index=True)
df = manhattan_venues[manhattan_venues['Venue Category'] == 'Pizza Place']
df['Borough'] = 'Manhattan'
df = df[columns]
pizzaPlaces = pizzaPlaces.append(df, ignore_index=True)
df = brooklyn_venues[brooklyn_venues['Venue Category'] == 'Pizza Place']
df['Borough'] = 'Brooklyn'
df = df[columns]
pizzaPlaces = pizzaPlaces.append(df, ignore_index=True)
df = queens_venues[queens_venues['Venue Category'] == 'Pizza Place']
df['Borough'] = 'Queens'
df = df[columns]
pizzaPlaces = pizzaPlaces.append(df, ignore_index=True)
df = staten_island_venues[staten_island_venues['Venue Category'] == 'Pizza Place']
df['Borough'] = 'Staten Island'
df = df[columns]
pizzaPlaces = pizzaPlaces.append(df, ignore_index=True)
pizzaPlaces

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,Borough,Neighborhood,Venue,Venue Latitude,Venue Longitude
0,Bronx,Co-op City,Capri II Pizza,40.876374,-73.829940
1,Bronx,Eastchester,Mario's Pizza,40.888628,-73.831260
2,Bronx,Kingsbridge,Kingsbridge Social Club,40.884545,-73.901964
3,Bronx,Kingsbridge,Sam's Pizza,40.879435,-73.905859
4,Bronx,Kingsbridge,Broadway Pizza & Pasta,40.878822,-73.904494
5,Bronx,Kingsbridge,Little Caesars Pizza,40.880002,-73.904140
6,Bronx,Kingsbridge,Domino's Pizza,40.884200,-73.902400
7,Bronx,Kingsbridge,Papa John's,40.884015,-73.903083
8,Bronx,Woodlawn,Katonah Pizza and Pasta,40.898784,-73.867457
9,Bronx,Woodlawn,Bella Napoli 2,40.896730,-73.862320


Now let's plot all the Pizza Places in the New York state to see where are all the Pizza places situated with respect to our sorted out areas based on common venues.

In [60]:
for lat, lng, borough, neighborhood, venue in zip(pizzaPlaces['Venue Latitude'], pizzaPlaces['Venue Longitude'], pizzaPlaces['Borough'], pizzaPlaces['Neighborhood'], pizzaPlaces['Venue']):
    label = '{}, {}, {}'.format(venue, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(newyork_map)  
    
newyork_map

Now let's plot a heat-map to identify the density of pizza places in entire New York state.

In [61]:
from folium import plugins
locationsArr = pizzaPlaces[['Venue Latitude', 'Venue Longitude']].as_matrix()
newyork_map.add_child(plugins.HeatMap(locationsArr, radius=15))
newyork_map

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In the above graph, green circles are the regions where our venue of concern, i.e., Pizza Place is not among the top 10 most common venues. And the red dots denote all the pizza places in the state of New York and a heat map is plotted on the map to show the density of Pizza Places in areas of New York.

Since we have many options, to narrow down our list, we will consider only the neighborhoods which have either no pizza place or just 1 pizza place in 1Km radius.

Since we have limited data, we will consider that all the neighborhoods are public zones and people live in those regions.

Based on the parameters defined and the heatmap, we can make a new dataframe which can be the best locations for our client to set up their new branch/outlet.

In [62]:
refinedSuggestedAreas = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'])

In [63]:
refinedVenues = getNearbyVenues(names=suggestedAreas['Neighborhood'],
                                   latitudes=suggestedAreas['Latitude'],
                                   longitudes=suggestedAreas['Longitude'],
                                   radius = 1000
                                  )

Baychester
Bronxdale
City Island
Clason Point
Country Club
Eastchester
Edenwald
Fieldston
Longwood
Mount Hope
Pelham Bay
Port Morris
Riverdale
Soundview
Spuyten Duyvil
Wakefield
Williamsbridge
Battery Park City
Central Harlem
Chelsea
Chinatown
Civic Center
East Harlem
Flatiron
Hudson Yards
Lincoln Square
Lower East Side
Manhattanville
Midtown South
Morningside Heights
Roosevelt Island
Stuyvesant Town
Tribeca
Tudor City
Turtle Bay
Upper East Side
Washington Heights
Bensonhurst
Bergen Beach
Boerum Hill
Brighton Beach
Broadway Junction
Brownsville
Canarsie
City Line
Coney Island
Dyker Heights
East Flatbush
East Williamsburg
Flatbush
Flatlands
Georgetown
Gowanus
Mill Island
Ocean Hill
Ocean Parkway
Paerdegat Basin
Prospect Heights
Remsen Village
Sea Gate
Vinegar Hill
Weeksville
Williamsburg
Windsor Terrace
Wingate
Arverne
Astoria
Astoria Heights
Auburndale
Bay Terrace
Bayswater
Beechhurst
Bellaire
Belle Harbor
Blissville
Breezy Point
Briarwood
Brookville
Cambria Heights
Douglaston
East Elm

Let's make a copy of our retrieved data to process.

In [64]:
refinedVenuesUsable = refinedVenues.copy(deep=True)

Let's add the borough names of each neighborhood to the dataFrame.

In [65]:
all_borough = []
for neigh in refinedVenuesUsable['Neighborhood']:
    df = suggestedAreas[suggestedAreas['Neighborhood'] == neigh]
    all_borough.append(df['Borough'].values[0])
refinedVenuesUsable.insert(0, 'Borough', all_borough)

In [66]:
refinedVenuesUsable.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Baychester,40.866858,-73.835798,Caridad & Louie,40.865843,-73.837707,Spanish Restaurant
1,Bronx,Baychester,40.866858,-73.835798,Panera Bread,40.867866,-73.827845,Bakery
2,Bronx,Baychester,40.866858,-73.835798,Fratelli's,40.863019,-73.843607,Italian Restaurant
3,Bronx,Baychester,40.866858,-73.835798,Four Seasons Nails,40.869285,-73.844468,Spa
4,Bronx,Baychester,40.866858,-73.835798,Nicks Pizza,40.870352,-73.846171,Pizza Place


Now we will divide all the possible identified neighborhoods into best, moderate and bad categories based on number of competitors of our client(i.e., Pizza Place Venue Category).

In [67]:
bestSuggested = pd.DataFrame(columns=['Borough','Neighborhood', 'Latitude', 'Longitude'])
moderateSuggested = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'])
worstSuggested = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'])
for neigh in refinedVenuesUsable['Neighborhood'].unique():
    df = refinedVenuesUsable[refinedVenuesUsable['Neighborhood'] == neigh]
    if df[df['Venue Category'] == 'Pizza Place'].shape[0] < 2:
        bestSuggested = bestSuggested.append({'Borough': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Borough'].values[0], 'Neighborhood': neigh, 'Latitude': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Latitude'].values[0], 'Longitude': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Longitude'].values[0]}, ignore_index=True)
    if df[df['Venue Category'] == 'Pizza Place'].shape[0] in range(2, 5):
        moderateSuggested = moderateSuggested.append({'Borough': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Borough'].values[0], 'Neighborhood': neigh, 'Latitude': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Latitude'].values[0], 'Longitude': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Longitude'].values[0]}, ignore_index=True)
    if df[df['Venue Category'] == 'Pizza Place'].shape[0] >=5:
        worstSuggested = worstSuggested.append({'Borough': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Borough'].values[0], 'Neighborhood': neigh, 'Latitude': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Latitude'].values[0], 'Longitude': suggestedAreas[suggestedAreas['Neighborhood'] == neigh]['Longitude'].values[0]}, ignore_index=True)

In [68]:
bestSuggested.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,City Island,40.847247,-73.786488
1,Bronx,Clason Point,40.806551,-73.854144
2,Bronx,Soundview,40.821012,-73.865746
3,Bronx,Wakefield,40.894705,-73.847201
4,Manhattan,Chelsea,40.744035,-74.003116


In [69]:
moderateSuggested.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Baychester,40.866858,-73.835798
1,Bronx,Country Club,40.844246,-73.824099
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Edenwald,40.884561,-73.848083
4,Bronx,Fieldston,40.895437,-73.905643


In [70]:
worstSuggested.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Bronxdale,40.852723,-73.861726
1,Bronx,Mount Hope,40.848842,-73.908299
2,Bronx,Williamsbridge,40.881039,-73.857446
3,Manhattan,East Harlem,40.792249,-73.944182
4,Manhattan,Stuyvesant Town,40.731000,-73.974052


Now we plot the best, moderate and worst places on a map to visualize our data.

In [71]:
suggestedMap = folium.Map(location=[latitude, longitude], zoom_start=10, width=900, height=500)
for lat, lng, borough, neighborhood in zip(bestSuggested['Latitude'], bestSuggested['Longitude'], bestSuggested['Borough'], bestSuggested['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=500,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=1,
        opacity=0.3,
        parse_html=False).add_to(suggestedMap)
for lat, lng, borough, neighborhood in zip(moderateSuggested['Latitude'], moderateSuggested['Longitude'], moderateSuggested['Borough'], moderateSuggested['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=500,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        opacity=0.3,
        parse_html=False).add_to(suggestedMap) 
for lat, lng, borough, neighborhood in zip(worstSuggested['Latitude'], worstSuggested['Longitude'], worstSuggested['Borough'], worstSuggested['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=500,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        opacity=0.3,
        parse_html=False).add_to(suggestedMap) 
    
suggestedMap

<h3>Now we will use <u>DBSCAN clustering</u> method to cluster all the clusters of <i>best</i> neighborhoods for our client.
    We do this to find the regions which do not have sufficient number of <i>Pizza Places</i> in a big area, thus increasing the chances of success for our client.</h3>

In [72]:
from sklearn.cluster import DBSCAN
coords = bestSuggested.as_matrix(columns=['Latitude', 'Longitude'])
db = DBSCAN(eps=0.0005, min_samples=3, metric='haversine', metric_params=None, algorithm='ball_tree',
          leaf_size=30, p=None, n_jobs=1).fit(np.radians(coords))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [73]:
clusters = db.labels_

In [74]:
bestSuggested.insert(0, 'Cluster Labels', db.labels_)
bestSuggested

,Cluster Labels,Borough,Neighborhood,Latitude,Longitude
0,-1,Bronx,City Island,40.847247,-73.786488
1,-1,Bronx,Clason Point,40.806551,-73.854144
2,-1,Bronx,Soundview,40.821012,-73.865746
3,-1,Bronx,Wakefield,40.894705,-73.847201
4,0,Manhattan,Chelsea,40.744035,-74.003116
5,0,Manhattan,Civic Center,40.715229,-74.005415
6,0,Manhattan,Flatiron,40.739673,-73.990947
7,0,Manhattan,Hudson Yards,40.756658,-74.000111
8,0,Manhattan,Lincoln Square,40.773529,-73.985338
9,0,Manhattan,Tribeca,40.721522,-74.010683


We will select only those rows which are classified among some cluster(i.e. cluster value is not -1)

In [75]:
from scipy import stats
clusters = clusters[clusters >= 0]

### Now we will plot all the clusters and all the suggested neighborhoods in the clusters on a map to show which areas are best for expansion for our client.

Make a list of all cluster centers and radii as well as all LatLng and label values of the neighborhoods in identified clusters.

In [76]:
circleRadius = []
circleLatLng = []
labels = []
allLat = []
allLng = []

In [77]:
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lat2, lon1, lon2): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

In [78]:
distances = []
for i in range(bestSuggested['Cluster Labels'].max() + 1):
    print('Cluster ', str(i + 1), ': ', str(bestSuggested[bestSuggested['Cluster Labels'] == i].count().values[0]))
    latitudes = np.array(bestSuggested[bestSuggested['Cluster Labels'] == i]['Latitude'].values)
    longitudes = np.array(bestSuggested[bestSuggested['Cluster Labels'] == i]['Longitude'].values)
    allLat.append(latitudes)
    allLng.append(longitudes)
    meanLat = latitudes.mean()
    meanLng = longitudes.mean()
    circleLatLng.append((meanLat, meanLng))
    clusterLabels = []
    for j in range(bestSuggested[bestSuggested['Cluster Labels'] == i].count().values[0]):
        distances.append(distance(latitudes[j], meanLat, longitudes[j], meanLng))
        clusterLabels.append(", ".join([bestSuggested[bestSuggested['Cluster Labels'] == i]['Neighborhood'].values[j], bestSuggested[bestSuggested['Cluster Labels'] == i]['Borough'].values[j]]))
    labels.append(clusterLabels)
    distances = np.array(distances)
    circleRadius.append(distances.max())
    distances = []

Cluster  1 :  11
Cluster  2 :  4
Cluster  3 :  4
Cluster  4 :  3
Cluster  5 :  11
Cluster  6 :  5


Plot the clusters and neighborhoods which are best suited for our client based on location, to establish their new outlet/branch.

In [79]:
finalMap = folium.Map(location=[latitude, longitude], zoom_start=10, width=900, height=500)
for i in range(len(circleRadius)):
    folium.Circle(
        [circleLatLng[i][0], circleLatLng[i][1]],
        radius=circleRadius[i]*1000,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.3,
        opacity=0.3,
        parse_html=False).add_to(finalMap)
finalMap

In [80]:
for i in range(len(labels)):
    for j in range(len(labels[i])):
        folium.Marker([allLat[i][j], allLng[i][j]], tooltip=labels[i][j], popup=labels[i][j], icon=folium.Icon(color='white', icon_color='green', icon='thumbs-o-up', prefix='fa')).add_to(finalMap)
finalMap

Now we find the biggest cluster of neighborhoods qualifying all out criterias to suggest as the best region for our client to establish their new branch/outlet.

In [81]:
mostDenseCluster = stats.mode(clusters).mode[0]

In [82]:
finalColumns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
bestSuggestedFinalPlaces = pd.DataFrame(columns=finalColumns)
df = bestSuggested[bestSuggested['Cluster Labels'] == mostDenseCluster]
for columnName in finalColumns:
    bestSuggestedFinalPlaces[columnName] = df[columnName].values
df2 = bestSuggested[bestSuggested['Cluster Labels'] != mostDenseCluster]
otherFinalPlaces = pd.DataFrame(columns=finalColumns)
for columnName in finalColumns:
    otherFinalPlaces[columnName] = df2[columnName].values

In [83]:
bestSuggestedFinalPlaces

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Chelsea,40.744035,-74.003116
1,Manhattan,Civic Center,40.715229,-74.005415
2,Manhattan,Flatiron,40.739673,-73.990947
3,Manhattan,Hudson Yards,40.756658,-74.000111
4,Manhattan,Lincoln Square,40.773529,-73.985338
5,Manhattan,Tribeca,40.721522,-74.010683
6,Manhattan,Turtle Bay,40.752042,-73.967708
7,Brooklyn,Boerum Hill,40.685683,-73.983748
8,Brooklyn,Vinegar Hill,40.703321,-73.981116
9,Queens,Hunters Point,40.743414,-73.953868


# Conclusion

From all the above data processing and analysis, we can come to a final conclusion that following neighborhoods are the best for our client to carry out further research on the likings and average money spent on eating out, by the residents, to further narrow down their new outlet location:

In [84]:
print(bestSuggestedFinalPlaces[['Neighborhood', 'Borough']].to_string(index=False))

   Neighborhood    Borough
        Chelsea  Manhattan
   Civic Center  Manhattan
       Flatiron  Manhattan
   Hudson Yards  Manhattan
 Lincoln Square  Manhattan
        Tribeca  Manhattan
     Turtle Bay  Manhattan
    Boerum Hill   Brooklyn
   Vinegar Hill   Brooklyn
  Hunters Point     Queens
     Ravenswood     Queens


Client may also want to look into following neighborhoods, although they do not make a huge cluster and thus may not be as beneficial for our client to invest their resources on:

In [85]:
print(otherFinalPlaces[['Neighborhood', 'Borough']].to_string(index=False))

        Neighborhood        Borough
         City Island          Bronx
        Clason Point          Bronx
           Soundview          Bronx
           Wakefield          Bronx
        Bergen Beach       Brooklyn
      Brighton Beach       Brooklyn
       East Flatbush       Brooklyn
         Mill Island       Brooklyn
            Sea Gate       Brooklyn
             Arverne         Queens
           Bayswater         Queens
        Belle Harbor         Queens
        Breezy Point         Queens
          Brookville         Queens
          Douglaston         Queens
           Glen Oaks         Queens
           Laurelton         Queens
            Neponsit         Queens
            Rosedale         Queens
             Roxbury         Queens
          Somerville         Queens
 Springfield Gardens         Queens
           Arlington  Staten Island
            Arrochar  Staten Island
          Bloomfield  Staten Island
        Butler Manor  Staten Island
          Charleston  Staten